In [6]:
import os
import numpy as np
import argparse
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
# import visdom
# import collections.abc
import collections
from utility import PGMdataset, RAVENdataset, ToTensor, dataset
from sran import SRAN



In [7]:
class args():
    model='SRAN'
    epochs=100
    batch_size=64
    seed=12345
    dataset = 'I-RAVEN'
    device = 2
    load_workers = 16
    resume = ''
    dataset_path = "/common/users/pv217/pritish_data"
    save = "/common/users/pv217/model_checkpoints/full_cnn_lstm"
    img_size = 64
    lr = 1e-4
    beta1 = 0.9
    beta2 = 0.999
    epsilon = 1e-8
    meta_alpha = 0.0
    meta_beta = 0.0
    cuda = torch.cuda.is_available()    
    visdom = False
    debug = False
    

In [8]:
# parser = argparse.ArgumentParser(description='our_model')
# parser.add_argument('--model', type=str, default='SRAN')
# parser.add_argument('--dataset', type=str, default='I-RAVEN', choices=['PGM', 'I-RAVEN'])
# parser.add_argument('--img_size', type=int, default=224)
# parser.add_argument('--epochs', type=int, default=200)
# parser.add_argument('--batch_size', type=int, default=32)
# parser.add_argument('--seed', type=int, default=12345)
# parser.add_argument('--load_workers', type=int, default=16)
# parser.add_argument('--resume', type=str, default='')
# #parser.add_argument('--dataset_path', type=str, default='/media/dsg3/datasets/PGM')
# parser.add_argument('--dataset_path', type=str, default='/media/dsg3/datasets/I-RAVEN')
# parser.add_argument('--save', type=str, default='/media/dsg3/hs')
# parser.add_argument('--lr', type=float, default=1e-4)
# parser.add_argument('--beta1', type=float, default=0.9)
# parser.add_argument('--beta2', type=float, default=0.999)
# parser.add_argument('--epsilon', type=float, default=1e-8)
# parser.add_argument('--meta_beta', type=float, default=0.0)
# parser.add_argument('--visdom', default=True, help='Use visdom for visualization')
# parser.add_argument('--cuda', default=True )
# parser.add_argument('--debug', default=False)



In [9]:
# args = parser.parse_args()
torch.cuda.set_device(args.device)
torch.cuda.manual_seed(args.seed)
args.save += '/' + args.dataset+'/'
start_time = time.strftime ('%Y-%m-%d_%H-%M-%S') 
if args.debug:
    args.save += args.dataset + '/'
    args.visdom = False
else:
    args.save += args.dataset +'_' + start_time + '/'
if not os.path.exists(args.save):
    os.makedirs(args.save)

In [10]:
args.train_figure_configurations = [0,1,2,3,4,5,6]
args.val_figure_configurations = args.train_figure_configurations
args.test_figure_configurations = [0,1,2,3,4,5,6]
train = dataset(args.dataset_path, "train", args.img_size, transform=transforms.Compose([ToTensor()]),shuffle=False,
                rotate=False,vertical_flip=False, vertical_roll = False,
                horizontal_flip = False, horizontal_roll= False, max_rotate_angle = 180
               )
valid = RAVENdataset(args.dataset_path, "val", args.val_figure_configurations, args.img_size, transform=transforms.Compose([ToTensor()]))
test = RAVENdataset(args.dataset_path, "test", args.test_figure_configurations, args.img_size, transform=transforms.Compose([ToTensor()]))

In [11]:
trainloader = DataLoader(train, batch_size=args.batch_size, shuffle=True, num_workers=args.load_workers)
validloader = DataLoader(valid, batch_size=args.batch_size, shuffle=False, num_workers=args.load_workers)
testloader = DataLoader(test, batch_size=args.batch_size, shuffle=False, num_workers=args.load_workers)

In [12]:
print ('Dataset:', args.dataset)
print ('Train/Validation/Test:{0}/{1}/{2}'.format(len(train), len(valid), len(test)))
print ('Image size:', args.img_size)

Dataset: I-RAVEN
Train/Validation/Test:42000/14000/14000
Image size: 64


In [13]:
if args.model == 'SRAN':
    model = SRAN(args)
start_epoch = 0
# if args.resume:
#     args.resume_epoch = 78
#     model.load_model(args.resume, args.resume_epoch)
#     print('Loaded model')
#     start_epoch = args.resume_epoch+1
# with open(os.path.join(args.save, 'results.log'), 'w') as f:
#     for key, value in vars(args).items():
#         f.write('{0}: {1}\n'.format(key, value))
#     f.write('--------------------------------------------------\n')

# pmodel = torch.nn.DataParallel(model)
# torch.backends.cudnn.benchmark = True
pmodel = model.cuda()

In [14]:
if args.visdom:
    viz = visdom.Visdom(port = 9527, env = args.dataset)

In [15]:
def train(epoch):
    model.train()
    train_loss = 0
    accuracy = 0
    loss_all = 0.0
    acc_all = 0.0
    counter = 0
    for batch_idx, (image, target, meta_target) in enumerate(trainloader):
        counter += 1
        if args.cuda:
            image = image.cuda()
#             print(1)
            target = target.cuda()
#             print(2)
            meta_target = meta_target.cuda()
#             print(3)
        model.optimizer.zero_grad()
        output = pmodel(image)
#         print(4)
        loss = model.compute_loss(output, target, meta_target)
        loss.backward()
        model.optimizer.step()
        pred = output[0].data.max(1)[1]
        correct = pred.eq(target.data).cpu().sum().numpy()
        accuracy = correct * 100. / target.size()[0]
        loss, acc = loss.item(), accuracy
#         print('Train: Epoch:{}, Batch:{}, Loss:{:.6f}, Acc:{:.4f}.'.format(epoch, batch_idx, loss, acc))
        loss_all += loss
        acc_all += acc
    if counter > 0:
        print("Avg Training Loss: {:.6f}".format(loss_all/float(counter)))
    return loss_all/float(counter), acc_all/float(counter)

In [16]:
def validate(epoch):
    model.eval()
    accuracy = 0
    acc_all = 0.0
    loss_all = 0.0
    counter = 0
    with torch.no_grad():
        for batch_idx, (image, target, meta_target) in enumerate(validloader):
            counter += 1
            if args.cuda:
                image = image.cuda()
                target = target.cuda()
                meta_target = meta_target.cuda()    
            output = pmodel(image)  
            loss = model.compute_loss(output, target, meta_target)
            pred = output[0].data.max(1)[1]
            correct = pred.eq(target.data).cpu().sum().numpy()
            accuracy = correct * 100. / target.size()[0]          
            acc = accuracy          
            acc_all += acc
            loss_all += loss.item()
    if counter > 0:
        print("Total Validation Acc: {:.4f}".format(acc_all/float(counter)))
    return loss_all/float(counter), acc_all/float(counter)

In [17]:
def test(epoch):
    model.eval()
    accuracy = 0
    acc_all = 0.0
    counter = 0
    with torch.no_grad():
        for batch_idx, (image, target, meta_target) in enumerate(testloader):
            counter += 1
            if args.cuda:
                image = image.cuda()
                target = target.cuda()
                meta_target = meta_target.cuda()   
            output = pmodel(image)
            pred = output[0].data.max(1)[1]
            correct = pred.eq(target.data).cpu().sum().numpy()
            accuracy = correct * 100. / target.size()[0]   
            acc = accuracy
            acc_all += acc
    if counter > 0:
        print("Total Testing Acc: {:.4f}".format(acc_all / float(counter)))
    return acc_all/float(counter)



In [ ]:
epoch_lst = []
train_loss_lst = []
train_acc_lst = []
val_loss_lst = []
val_acc_lst = []
test_acc_lst = []


for epoch in range(start_epoch, args.epochs):
    train_loss, train_acc = train(epoch)
    val_loss, val_acc = validate(epoch)
    test_acc = test(epoch)
#     model.save_model(args.save, epoch)
    epoch_lst.append(epoch)
    train_loss_lst.append(train_loss)
    train_acc_lst.append(train_acc)
    val_loss_lst.append(val_loss)
    val_acc_lst.append(val_acc)
    test_acc_lst.append(test_acc)

Avg Training Loss: 1.912737
Total Validation Acc: 29.9895
Total Testing Acc: 30.2916
Avg Training Loss: 1.596052
Total Validation Acc: 36.5654
Total Testing Acc: 37.9733
Avg Training Loss: 1.399158
Total Validation Acc: 37.9257
Total Testing Acc: 39.3550
Avg Training Loss: 1.190943
Total Validation Acc: 40.1898
Total Testing Acc: 41.2695
Avg Training Loss: 0.950413
Total Validation Acc: 43.1483
Total Testing Acc: 44.3993
Avg Training Loss: 0.768732
Total Validation Acc: 43.9498
Total Testing Acc: 44.3113
Avg Training Loss: 0.619694
Total Validation Acc: 44.1947
Total Testing Acc: 44.3184
Avg Training Loss: 0.505195
Total Validation Acc: 44.9320
Total Testing Acc: 44.2375
Avg Training Loss: 0.421054
Total Validation Acc: 44.2138
Total Testing Acc: 43.9046
Avg Training Loss: 0.370982
Total Validation Acc: 45.2197
Total Testing Acc: 44.6680
Avg Training Loss: 0.322922
Total Validation Acc: 44.6133
Total Testing Acc: 45.9784
Avg Training Loss: 0.293531
Total Validation Acc: 44.8725
Total T

In [ ]:
import pandas as pd
import seaborn as sns

In [ ]:
training_stats = pd.DataFrame({'epoch':epoch_lst,
 'training_loss':train_loss_lst,
 'training_accuracy':train_acc_lst,
 'validation_loss':val_loss_lst,
 'validation_accuracy':val_acc_lst,
 'test_accuracy':test_acc_lst
})

In [ ]:
training_stats = training_stats.set_index('epoch')

In [ ]:
sns.set(rc = {'figure.figsize':(15,8)})

In [ ]:
sns.lineplot(data=training_stats[['training_loss','validation_loss']])

In [ ]:
sns.lineplot(data=training_stats[['training_accuracy','validation_accuracy','test_accuracy']])

In [20]:
training_stats.tail(1)

,training_loss,training_accuracy,validation_loss,validation_accuracy,test_accuracy
epoch,,,,,
99,0.064734,98.0875,3.869491,44.5625,45.275


In [21]:
training_stats.sort_values('validation_accuracy', ascending=False).head()

,training_loss,training_accuracy,validation_loss,validation_accuracy,test_accuracy
epoch,,,,,
81,0.076495,97.566667,3.697726,45.8750,46.8750
98,0.068795,97.904167,4.328713,45.4125,45.1750
87,0.067424,97.904167,4.384807,45.2250,45.2500
92,0.071118,97.829167,4.481265,45.0750,45.7375
77,0.095982,96.995833,3.672894,45.0625,45.0125


In [ ]:
def main():

    
def create_vis_plot(_xlabel, _ylabel, _title, _legend):
    return viz.line(
        X=torch.zeros((1,)),
        Y=torch.zeros((1, 3)),
        opts=dict(
            xlabel=_xlabel,
            ylabel=_ylabel,
            title=_title,
            legend=_legend
        )
    )
if __name__ == '__main__':
    main()
